In [ ]:
!pip install geopandas shapely networkx tqdm


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Streets.shx to Streets.shx
Saving Streets.shp to Streets.shp
Saving Streets.sbx to Streets.sbx
Saving Streets.sbn to Streets.sbn
Saving Streets.prj to Streets.prj
Saving Streets.dbf to Streets.dbf
Saving Streets.cpg to Streets.cpg


In [ ]:
import geopandas as gpd

# Load your road shapefile
gdf = gpd.read_file("/content/Streets.shp")

# 🔐 Ensure CRS is WGS84 (EPSG:4326 - lat/lon)
if gdf.crs is None:
    print("⚠️ No CRS detected! Setting to EPSG:4326 (WGS84)...")
    gdf.set_crs(epsg=4326, inplace=True)
elif gdf.crs.to_epsg() != 4326:
    print(f"🔄 Converting CRS from {gdf.crs} to EPSG:4326...")
    gdf = gdf.to_crs(epsg=4326)
else:
    print("✅ CRS is already EPSG:4326 (WGS84)")

# Optional preview
gdf.head()

✅ CRS is already EPSG:4326 (WGS84)


,OBJECTID,LINK_ID,ST_NAME,geometry
0,1,590186215.0,Rīga-Stockholm,"LINESTRING (24.02475 57.06223, 24.00883 57.070..."
1,2,749463153.0,Ventspils-Nynäshamn,"LINESTRING (21.54389 57.39564, 21.53406 57.401..."
2,3,750303390.0,Rīgas iela,"LINESTRING (26.29975 56.96972, 26.29918 56.9699)"
3,4,750303392.0,Brūža iela,"LINESTRING (27.05029 57.42628, 27.05024 57.426..."
4,5,750303393.0,Tālavijas iela,"LINESTRING (27.72696 56.54542, 27.72673 56.54668)"


from shapely.geometry import box
import numpy as np

minx, miny, maxx, maxy = gdf.total_bounds
tile_size = 0.01  # Reduce for more zoom-in

grid_cells = []
x = minx
while x < maxx:
    y = miny
    while y < maxy:
        grid_cells.append(box(x, y, x + tile_size, y + tile_size))
        y += tile_size
    x += tile_size

grid = gpd.GeoDataFrame({'geometry': grid_cells})
grid.set_crs("EPSG:4326", inplace=True)


In [ ]:
from shapely.geometry import box
import numpy as np

minx, miny, maxx, maxy = gdf.total_bounds
tile_size = 0.01  # Reduce for more zoom-in

grid_cells = []
x = minx
while x < maxx:
    y = miny
    while y < maxy:
        grid_cells.append(box(x, y, x + tile_size, y + tile_size))
        y += tile_size
    x += tile_size

grid = gpd.GeoDataFrame({'geometry': grid_cells})
grid.set_crs("EPSG:4326", inplace=True)


,geometry
0,"POLYGON ((18.08297 55.67516, 18.08297 55.68516..."
1,"POLYGON ((18.08297 55.68516, 18.08297 55.69516..."
2,"POLYGON ((18.08297 55.69516, 18.08297 55.70516..."
3,"POLYGON ((18.08297 55.70516, 18.08297 55.71516..."
4,"POLYGON ((18.08297 55.71516, 18.08297 55.72516..."
...,...
280687,"POLYGON ((28.24297 58.38516, 28.24297 58.39516..."
280688,"POLYGON ((28.24297 58.39516, 28.24297 58.40516..."
280689,"POLYGON ((28.24297 58.40516, 28.24297 58.41516..."
280690,"POLYGON ((28.24297 58.41516, 28.24297 58.42516..."


In [ ]:
import networkx as nx
from shapely.geometry import LineString, Point, Polygon
from tqdm import tqdm
import math

def is_roundabout_like(loop, threshold=0.3):
    try:
        poly = Polygon(loop)
        if not poly.is_valid:
            return False
        area = poly.area
        perimeter = poly.length
        if perimeter == 0:
            return False
        compactness = (4 * math.pi * area) / (perimeter ** 2)
        return compactness > threshold
    except:
        return False

roundabout_results = []

start_tile_index = 0  # Resume from here if interrupted

for i, tile in tqdm(grid.iloc[start_tile_index:].iterrows(), total=len(grid) - start_tile_index):
    actual_i = i + start_tile_index
    clipped = gdf[gdf.intersects(tile.geometry)]
    if clipped.empty:
        continue

    G = nx.Graph()
    for _, row in clipped.iterrows():
        line = row.geometry
        if isinstance(line, LineString):
            coords = list(line.coords)
            for j in range(len(coords) - 1):
                G.add_edge(coords[j], coords[j + 1])

    loops = list(nx.cycle_basis(G))
    roundabout_candidates = [loop for loop in loops if is_roundabout_like(loop)]

    for loop in roundabout_candidates:
        try:
            poly = Polygon(loop)
            if poly.is_valid:
                roundabout_results.append({
                    "tile_id": actual_i,
                    "centroid_lat": poly.centroid.y,
                    "centroid_lon": poly.centroid.x,
                    "geometry": poly.wkt
                })
        except:
            continue


100%|██████████| 280692/280692 [2:40:08<00:00, 29.21it/s]


In [ ]:
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point, Polygon

# Convert detection results to GeoDataFrame
roundabout_geoms = []
tile_ids = []
centroids = []

for result in roundabout_results:
    geom = Polygon(wkt.loads(result["geometry"]))
    roundabout_geoms.append(geom)
    tile_ids.append(result["tile_id"])
    centroids.append(Point(result["centroid_lon"], result["centroid_lat"]))

# Create GeoDataFrame
roundabout_gdf = gpd.GeoDataFrame({
    "tile_id": tile_ids,
    "centroid": centroids
}, geometry=roundabout_geoms, crs="EPSG:4326")

# Save as Shapefile
roundabout_gdf.to_file("detected_roundabouts.shp")

# Download all components of shapefile
from google.colab import files
for ext in ['shp', 'shx', 'dbf', 'prj']:
    files.download(f"detected_roundabouts.{ext}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>